## 구글에 이미지 검색해서 저장하는 과정 자동화 하기
- 동적 크롤링 : selenium 이용
  - 코랩 selenium은 창이 뜨지 않음을 주의
- 이미지 저장 : src를 이용한다
  - get_atrribute이용해서 src 주소를 갖고옴
  - urllib.request.urlretrieve(이미지 src 주소, "저장할이름.jpg") 
- 항목선택 : f12 눌러서 class든 name이든 아무거나 하면 된다
  - class 경우는 그냥 이름이 rabbit이런식이여도 앞에 .을 붙여야
    - '.rabbit' 이런식으로
  - element : 항목 한개
  - elements : 항목 여러개 선택해서 리스트 담는 경우
    - by css : name이든 class든 다 되는듯 
- 영상참고 : https://www.youtube.com/watch?v=1b7pXC1-IbE 
- class name으로 저장하는 경우 웬만하면 다 이상없긴한데 꽤 종종 중복인경우가 있어서 이상한거까지 저장되는 경우가 있음
  - f12에서 마우스 우클릭 - copy full xpath 
  - 그리고 by css 말고 by xpath로!
- **코드가 정상적인데 오류 생기는경우는 time.sleep으로 쉬는 시간을 주자!!**

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # 당연하지만 구글 콜랩은 웹브라우저 창을 직접 띄어서 제어할 수 없기 때문에 headless(창이 안보이는 형태)로 제어합니다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [10]:
elem = driver.find_element_by_name("q") #name,class등 여러개로 제어할 수 있는데 여기서는 q라는 name으로 된 항목을 "선택"하겠단것 : <검색창 선택>
# element_by_css가 name이든 class든 다 되는듯
elem.send_keys("조코딩") # <검색창에 '조코딩'이라고 입력>
elem.send_keys(Keys.RETURN) # <엔터역할>

########################## 스크롤 내리는 코드 ##########################
last_height = driver.execute_script("return document.body.scrollHeight") # 전체 스크롤 높이 길이를 last_height에 저장
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    SCROLL_PAUSE_TIME = 1
    time.sleep(SCROLL_PAUSE_TIME) # 스크롤 사이마다 1초씩 쉬어준다
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height: # 이건 다 내렸다는 의미 : 구글에선 '결과 더보기'가 이 때나옴. 이걸 눌러서 또 더 내리겠단거
        try:
            driver.find_element_by_css_selector(".mye4qd").click() # 결과 더보기 클릭
        except:
            break
    last_height = new_height

########################## 이미지 선택 및 저장 ##########################
images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd") # 여러개 선택해서 리스트에 담으려면 elements! -> images 라는 커다란 리스트
count = 1
for image in images:
    try:
        image.click()
        time.sleep(2)
        imgUrl = driver.find_element_by_xpath('/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute("src")
        opener=urllib.request.build_opener()
        opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(imgUrl, str(count) + ".jpg") # 그냥 저장명 앞에 번호매겨줄려고 count 이용
        count = count + 1
    except:
        pass

driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


- 이미지를 다운로드 하실때 나무위키 등 특정 사이트의 경우 봇이 접근하는 것을 차단해서 urllib.request.urlretrieve(imgUrl, "test.jpg") 줄에서
 urllib.error.HTTPError: HTTP Error 403: Forbidden가 뜨실 수 있습니다.
- 그럴때는 아래와 같이 브라우저 인것 처럼 속이는 header를 추가하시면 정상적으로 크롤링 할 수 있습니다!

- opener=urllib.request.build_opener()
- opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64 AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
- urllib.request.install_opener(opener)
- urllib.request.urlretrieve(imgUrl, "test.jpg")